In [1]:
from pathlib import Path

import numpy
import pint
from util import p_

import autochem as ac
import automech
from autochem.util import plot

kB = pint.Quantity("boltzmann_constant").m_as("J/K")


class Key:
    allyl = "allyl"
    alkyl = "alkyl"
    vinyl = "vinyl"


eq_dct = {
    Key.allyl: "C5H8(522) + OH(4) = C5H7(500) + H2O(5)",
    Key.alkyl: "C5H8(522) + OH(4) = C5H7(504) + H2O(5)",
    Key.vinyl: "C5H8(522) + OH(4) = C5H7(1202) + H2O(5)",
}

exp_data_dct = {
    Key.allyl: {
        "order": 2,
        "A": 2.55e-11,
        "b": 0,
        "E": 1560 * kB,
        "units": {"substance": "molec", "energy": "J"},
    },
    Key.alkyl: {
        "order": 2,
        "A": 1.90e-11,
        "b": 0,
        "E": 1705 * kB,
        "units": {"substance": "molec", "energy": "J"},
    },
    Key.vinyl: {
        "order": 2,
        "A": 1.39e-11,
        "b": 0,
        "E": 2270 * kB,
        "units": {"substance": "molec", "energy": "J"},
    },
}

tag = "A_rh-oh_1e-1_v0"
root_path = Path("..") / Path("..")

In [2]:
par_mech = automech.io.read(p_.parent_mechanism("json", path=p_.data(root_path)))
cal_mech = automech.io.read(
    p_.calculated_mechanism(tag, "json", path=p_.data(root_path))
)

In [3]:
T_range=(856, 1258)
T = numpy.linspace(*T_range, 1000)
ks_lst = []
cal_rate_dct = {}

for key in (Key.allyl, Key.alkyl, Key.vinyl):
    print(key)

    cal_rxns = automech.reaction_rate_objects(cal_mech, eq_dct.get(key))
    par_rxns = automech.reaction_rate_objects(par_mech, eq_dct.get(key))
    exp_rate = ac.rate.data.ArrheniusRateFit.model_validate(exp_data_dct.get(key))

    assert len(cal_rxns) == 1, cal_rxns
    (cal_rxn,) = cal_rxns
    cal_rate = cal_rxn.rate
    order = cal_rate.order

    cal_rate_dct[key] = cal_rate
    rates = [cal_rate, exp_rate]
    labels = ["This work", "KAUST"]
    colors = [plot.Color.blue, plot.Color.brown]

    if par_rxns:
        assert len(par_rxns) == 1, par_rxns
        (par_rxn,) = par_rxns
        par_rate = par_rxn.rate
        rates.append(par_rate)
        labels.append("Hill")
        colors.append(plot.Color.red)

    ks = [r(T, P=1) for r in rates]
    ks_lst.append(ks)
    chart = plot.arrhenius(ks=ks, T=T, order=order, labels=labels, colors=colors)

    display(chart)

    chart.save(f"{key}.png", ppi=300)

allyl


alt.Chart(...)

alkyl


alt.Chart(...)

vinyl


alt.Chart(...)

In [4]:
key = "total"
k_cal = sum(ks[0] for ks in ks_lst)
k_exp = sum(ks[1] for ks in ks_lst)
ks = [k_cal, k_exp]
labels = ["This work", "KAUST"]
colors = [plot.Color.blue, plot.Color.brown]
chart = plot.arrhenius(ks=ks, T=T, order=order, labels=labels, colors=colors)

print(key)
display(chart)
chart.save(f"{key}.png", ppi=300)

total


alt.Chart(...)

In [7]:
def print_arrhenius_parameters(rate, name):
    assert isinstance(rate, ac.rate.data.ArrheniusRateFit)
    print(f"{name}\t{rate.A:.4e}\t{rate.b:.4f}\t{rate.E:.2f}")

for key, rate in cal_rate_dct.items():
    eq = eq_dct.get(key)
    print_arrhenius_parameters(rate, f"{eq}\t{key} abstraction")

C5H8(522) + OH(4) = C5H7(500) + H2O(5)	allyl abstraction	2.6750e+06	2.0410	-2679.00
C5H8(522) + OH(4) = C5H7(504) + H2O(5)	alkyl abstraction	1.1620e+05	2.3530	-2328.00
C5H8(522) + OH(4) = C5H7(1202) + H2O(5)	vinyl abstraction	9.8040e+02	2.9640	758.00
